In [ ]:
!zenml integration install sklearn mlflow -y

import IPython

# automatically restart kernel
IPython.Application.instance().kernel.do_shutdown(restart=True)

In [1]:
!zenml init

Found existing ZenML repository at path 
'/Users/strickvl/coding/zenml/repos/zenml/examples/quickstart/new_quickstart'.
⠋ Initializing ZenML repository at 
/Users/strickvl/coding/zenml/repos/zenml/examples/quickstart/new_quickstart.
⠋ Initializing ZenML repository at 
/Users/strickvl/coding/zenml/repos/zenml/examples/quickstart/new_quickstart.



In [2]:
# Register the MLflow experiment tracker
!zenml experiment-tracker register mlflow_tracker --flavor=mlflow

# Register the MLflow model registry
!zenml model-registry register mlflow_registry --flavor=mlflow

# Register the MLflow model deployer
!zenml model-deployer register mlflow_deployer --flavor=mlflow

# Register a new stack with the new stack components
!zenml stack register quickstart_stack -a default\
                                       -o default\
                                       -d mlflow_deployer\
                                       -e mlflow_tracker\
                                       -r mlflow_registry\

!zenml stack set quickstart_stack

Connected to the ZenML server: 'http://127.0.0.1:8237'
Running with active workspace: 'default' (repository)
Active repository stack set to: 'quickstart_stack'
⠋ Setting the repository active stack to 'quickstart_stack'...kstart_stack'...


In [42]:
!zenml stack describe

Connected to the ZenML server: 'http://127.0.0.1:8237'
Running with active workspace: 'default' (repository)
          Stack Configuration           
┏━━━━━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━┓
┃ COMPONENT_TYPE     │ COMPONENT_NAME  ┃
┠────────────────────┼─────────────────┨
┃ EXPERIMENT_TRACKER │ mlflow_tracker  ┃
┠────────────────────┼─────────────────┨
┃ MODEL_DEPLOYER     │ mlflow_deployer ┃
┠────────────────────┼─────────────────┨
┃ ORCHESTRATOR       │ default         ┃
┠────────────────────┼─────────────────┨
┃ ARTIFACT_STORE     │ default         ┃
┠────────────────────┼─────────────────┨
┃ MODEL_REGISTRY     │ mlflow_registry ┃
┗━━━━━━━━━━━━━━━━━━━━┷━━━━━━━━━━━━━━━━━┛
   'quickstart_stack' stack (ACTIVE)    
⠋ Describing stack...
Stack 'quickstart_stack' with id '10db1c10-a6f4-43ea-977a-3dfedb3b2fed' is owned
by user default and is 'private'.
⠋ Describing stack...
⠋ Describing stack...



In [43]:
import pandas as pd
from sklearn.model_selection import train_test_split

from zenml import step
from zenml.steps import Output


@step(enable_cache=True)
def training_data_loader() -> (
    Output(
        X_train=pd.DataFrame,
        X_test=pd.DataFrame,
        y_train=pd.Series,
        y_test=pd.Series,
    )
):
    """Load the Census Income dataset as tuple of Pandas DataFrame / Series."""
    # Load the dataset
    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
    column_names = [
        "age",
        "workclass",
        "fnlwgt",
        "education",
        "education-num",
        "marital-status",
        "occupation",
        "relationship",
        "race",
        "sex",
        "capital-gain",
        "capital-loss",
        "hours-per-week",
        "native-country",
        "income",
    ]
    data = pd.read_csv(
        url, names=column_names, na_values="?", skipinitialspace=True
    )

    # Drop rows with missing values
    data = data.dropna()

    # Encode categorical features and drop original columns
    categorical_cols = [
        "workclass",
        "education",
        "marital-status",
        "occupation",
        "relationship",
        "race",
        "sex",
        "native-country",
    ]
    data = pd.get_dummies(data, columns=categorical_cols, drop_first=True)

    # Encode target feature
    data["income"] = data["income"].apply(
        lambda x: 1 if x.strip() == ">50K" else 0
    )

    # Separate features and target
    X = data.drop("income", axis=1)
    y = data["income"]

    # Split the dataset into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
    return X_train, X_test, y_train, y_test

In [44]:
X_train, X_test, y_train, y_test = training_data_loader()

In [45]:
X_train

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,workclass_Local-gov,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,...,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia
19863,53,168539,5,0,0,70,False,False,False,True,...,False,False,False,False,False,False,False,True,False,False
24342,49,56841,13,0,0,70,False,False,False,True,...,False,False,False,False,False,False,False,True,False,False
10027,28,154571,10,0,0,40,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
25710,60,188236,6,0,0,40,False,True,False,False,...,False,False,False,False,False,False,False,True,False,False
13824,53,87158,9,0,0,40,False,True,False,False,...,False,False,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32171,40,67852,9,0,0,35,False,True,False,False,...,False,False,False,False,False,False,False,True,False,False
5875,41,120539,10,3103,0,40,False,False,False,True,...,False,False,False,False,False,False,False,True,False,False
935,37,176900,9,0,0,99,False,True,False,False,...,False,False,False,False,False,False,False,True,False,False
17056,56,51662,7,0,0,40,False,False,False,True,...,False,False,False,False,False,False,False,True,False,False


In [46]:
import mlflow

from sklearn.base import ClassifierMixin
from sklearn.ensemble import RandomForestClassifier

from zenml.client import Client

experiment_tracker = Client().active_stack.experiment_tracker


@step(enable_cache=True, experiment_tracker=experiment_tracker.name)
def random_forest_trainer_mlflow(
    X_train: pd.DataFrame,
    y_train: pd.Series,
) -> RandomForestClassifier:
    """Train a sklearn Random Forest classifier and log to MLflow."""
    mlflow.sklearn.autolog()  # log all model hparams and metrics to MLflow
    model = RandomForestClassifier()
    model.fit(X_train.to_numpy(), y_train.to_numpy())
    train_acc = model.score(X_train.to_numpy(), y_train.to_numpy())
    print(f"Train accuracy: {train_acc}")
    return model

Reloading configuration file /Users/strickvl/coding/zenml/repos/zenml/examples/quickstart/new_quickstart/.zen/config.yaml


In [47]:
from sklearn.linear_model import SGDClassifier


@step(enable_cache=True, experiment_tracker=experiment_tracker.name)
def sgd_trainer_mlflow(
    X_train: pd.DataFrame,
    y_train: pd.Series,
) -> ClassifierMixin:
    """Train a SGD classifier and log to MLflow."""
    mlflow.sklearn.autolog()  # log all model hparams and metrics to MLflow
    model = SGDClassifier()
    model.fit(X_train.to_numpy(), y_train.to_numpy())
    train_acc = model.score(X_train.to_numpy(), y_train.to_numpy())
    print(f"Train accuracy: {train_acc}")
    return model

In [48]:
@step
def evaluator(
    X_test: pd.DataFrame,
    y_test: pd.Series,
    model1: ClassifierMixin,
    model2: ClassifierMixin,
) -> ClassifierMixin:
    """Calculate the accuracy on the test set and return the best model of two."""
    test_acc1 = model1.score(X_test.to_numpy(), y_test.to_numpy())
    test_acc2 = model2.score(X_test.to_numpy(), y_test.to_numpy())
    print(f"Test accuracy ({model1.__class__.__name__}): {test_acc1}")
    print(f"Test accuracy ({model2.__class__.__name__}): {test_acc2}")
    return model1 if test_acc1 > test_acc2 else model2

In [49]:
from zenml import pipeline
from zenml.integrations.mlflow.steps.mlflow_registry import (
    mlflow_register_model_step,
)
from zenml.model_registries.base_model_registry import (
    ModelRegistryModelMetadata,
)


@pipeline(enable_cache=True)
def train_and_register_model_pipeline() -> ClassifierMixin:
    """Train a model."""
    X_train, X_test, y_train, y_test = training_data_loader()
    model1 = random_forest_trainer_mlflow(X_train=X_train, y_train=y_train)
    model2 = sgd_trainer_mlflow(X_train=X_train, y_train=y_train)
    best_model = evaluator(
        X_test=X_test, y_test=y_test, model1=model1, model2=model2
    )
    mlflow_register_model_step.with_options(
        parameters=dict(
            name="zenml-quickstart-model",
            metadata=ModelRegistryModelMetadata(
                arch=f"{best_model.__class__.__name__}"
            ),
            description="The first run of the Quickstart pipeline.",
        )
    )(best_model)
    return best_model


train_and_register_model_pipeline()

Registered pipeline train_and_register_model_pipeline (version 2).
Running pipeline train_and_register_model_pipeline on stack quickstart_stack (caching enabled)
Step training_data_loader has started.
Step training_data_loader has finished in 19.506s.
Step random_forest_trainer_mlflow has started.


2023/06/13 17:32:32 INFO mlflow.tracking.fluent: Experiment with name 'train_and_register_model_pipeline' does not exist. Creating a new experiment.
2023/06/13 17:32:32 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.


Train accuracy: 1.0
Step random_forest_trainer_mlflow has finished in 12.507s.
Step sgd_trainer_mlflow has started.


2023/06/13 17:32:45 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.


Train accuracy: 0.7846160222139335
Step sgd_trainer_mlflow has finished in 7.071s.
Step evaluator has started.
Test accuracy (RandomForestClassifier): 0.849494447207028
Test accuracy (SGDClassifier): 0.7853472567545168
Step evaluator has finished in 1.195s.
Step mlflow_register_model_step has started.
MLflow model registry does not take a version as an argument. Registering a new version for the model 'zenml-quickstart-model' a version will be assigned automatically.


2023/06/13 17:32:55 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: zenml-quickstart-model, version 2


Registered model zenml-quickstart-model with version 2 from source file:///Users/strickvl/Library/Application Support/zenml/local_stores/c822eaea-b94a-442a-8909-bb44dac02eb6/mlruns/739635418256350737/c1b2ef27e867482193737b6c7456e4a7/artifacts/model.
Step mlflow_register_model_step has finished in 0.679s.
Pipeline run train_and_register_model_pipeline-2023_06_13-15_32_12_152102 has finished in 43.578s.
Dashboard URL: http://127.0.0.1:8237/workspaces/default/pipelines/95e17c8c-46e2-4d5c-8e02-28dbe8a41651/runs


In [50]:
@pipeline
def register_and_deploy_model() -> None:
    """Print the name of the model."""
    best_model = train_and_register_model_pipeline()


In [51]:
register_and_deploy_model()

Registered pipeline register_and_deploy_model (version 4).
Running pipeline register_and_deploy_model on stack quickstart_stack (caching enabled)
Step training_data_loader has started.
Using cached version of training_data_loader.
Step random_forest_trainer_mlflow has started.
Using cached version of random_forest_trainer_mlflow.
Step sgd_trainer_mlflow has started.
Using cached version of sgd_trainer_mlflow.
Step evaluator has started.
Using cached version of evaluator.
Step mlflow_register_model_step has started.
Using cached version of mlflow_register_model_step.
Pipeline run register_and_deploy_model-2023_06_13-15_32_57_790478 has finished in 2.958s.
Dashboard URL: http://127.0.0.1:8237/workspaces/default/pipelines/73ab8a51-e9fd-4a52-be93-0ca0e5c8d90c/runs
